<a href="https://colab.research.google.com/github/NChanko/medical_ner/blob/main/Medical_NER_NCKK_evaluating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install spacy[transformers]

#OLD TEST

In [ ]:
#REF https://newscatcherapi.com/blog/train-custom-named-entity-recognition-ner-model-with-spacy-v3

#https://ner.pythonhumanities.com/03_02_train_spacy_ner_model.html

In [ ]:
import json

with open('data.json', 'r') as f:
    data = json.load(f)

print(data['examples'][0])

In [ ]:
training_data = {'classes' : ['MEDICINE', "MEDICALCONDITION", "PATHOGEN"], 'annotations' : []}
for example in data['examples']:
  temp_dict = {}
  temp_dict['text'] = example['content']
  temp_dict['entities'] = []
  for annotation in example['annotations']:
    start = annotation['start']
    end = annotation['end']
    label = annotation['tag_name'].upper()
    temp_dict['entities'].append((start, end, label))
  training_data['annotations'].append(temp_dict)

print(training_data['annotations'][0])

{'text': "While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]", 'entities': [(360, 371, 'MEDICINE'), (383, 408, 'MEDICINE'), (104, 112, 'MEDICALCONDITION'), (679,

In [ ]:
df = pd.read_csv("trained_data.csv")
new_df=df['train_set']
new_df

0       {'text': 'SUBJECTIVE:,  This 23-year-old white...
1       {'text': 'PAST MEDICAL HISTORY:, He has diffic...
2       {'text': 'HISTORY OF PRESENT ILLNESS: , I have...
3       {'text': '2-D M-MODE: , ,1.  Left atrial enlar...
4       {'text': '1.  The left ventricular cavity size...
                              ...                        
4994    {'text': "HISTORY:,  I had the pleasure of mee...
4995    {'text': 'ADMITTING DIAGNOSIS: , Kawasaki dise...
4996    {'text': 'SUBJECTIVE: , This is a 42-year-old ...
4997    {'text': 'CHIEF COMPLAINT: , This 5-year-old m...
4998    {'text': 'HISTORY: , A 34-year-old male presen...
Name: train_set, Length: 4999, dtype: object

In [ ]:
import spacy
import pandas as pd
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin() # create a DocBin object

In [ ]:
from spacy.util import filter_spans

#for training_example  in tqdm(training_data['annotations']):
for training_example  in tqdm(new_df):

    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy") # save the docbin object

In [ ]:
!python -m spacy init fill-config "/content/base_config.cfg" config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0


ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-12-18 22:18:21,331] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-12-18 22:18:21,342] [INFO] Pipeline: ['transformer', 'ner']
INFO:spacy:Pipeline: ['transformer', 'ner']
[2022-12-18 22:18:21,345] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-12-18 22:18:21,346] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining m

In [ ]:
#To move the folder from Google Colab to Google Drive. Try this tip.
# First zip the folder with foloowing code.
!zip -r /content/model-best.zip /content/model-best
from google.colab import files
files.download("model-best.zip")

#The next step is to set the colab link and Gdrive link to copy and paste. Don't forget to enable Gdrie.

In [ ]:
import shutil
colab_link = "/content/model-best.zip"
gdrive_link = "/content/drive/MyDrive/Colab Notebooks/Models"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/Colab Notebooks/Models/model-best.zip'

#Testing with custom model

In [ ]:
import pandas as pd
import spacy
doc= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP NER Project/mtsamples.csv")

doc_df = pd.DataFrame(doc)


In [ ]:
note ="""HPI: -- is a 76 yo man with h/o HTN, DM, and sleep apnea who presented to the ED
complaining of chest pain. He states that the pain began the day before and consisted of a
sharp pain that lasted around 30 seconds, followed by a dull pain that would last around 2
minutes. The pain was located over his left chest area somewhat near his shoulder. The
onset of pain came while the patient was walking in his home. He did not sit and rest
during the pain, but continued to do household chores. Later on in the afternoon he went
to the gym where he walked 1 mile on the treadmill, rode the bike for 5 minutes, and
swam in the pool. After returning from the gym he did some work out in the yard,
cutting back some vines. He did not have any reoccurrences of chest pain while at the
gym or later in the evening. The following morning (of his presentation to the ED) he
noticed the pain as he was getting out of bed. Once again it was a dull pain, preceded by
a short interval of a sharp pain. The patient did experience some tingling in his right arm
after the pain ceased. He continued to have several episodes of the pain throughout the
morning, so his daughter-in-law decided to take him to the ED around 12:30pm. The
painful episodes did not increase in intensity or severity during this time. At the ED the
patient was given nitroglycerin, which he claims helped alleviate the pain somewhat. --
has not experienced any shortness of breath, nausea, or diaphoresis during these episodes
of pain. He has never had chest pain in the past. He has been told “years ago” that he
has a right bundle branch block and premature heart beats. """

In [ ]:
cmed_ner_nckk = spacy.load("/content/drive/MyDrive/Colab Notebooks/NLP NER Project/Models/cmed_ner_nckk/model_best")
cmed_ner_nckk.component_names
cmed_ner_nckk.get_pipe('ner').labels

('DISEASE', 'DRUG', 'LAB', 'SYMPTOM')

In [ ]:
def is_negated(entity, text):
    # Define a list of negation keywords
    negation_keywords = ['no', 'without', 'denies', 'negative', 'not present']

    # Check if any of the negation keywords appear in the context around the entity
    for keyword in negation_keywords:
        if keyword in text[max(0, entity.start_char-20):entity.start_char]:
            return True
    return False

In [ ]:
#doc = nlp_ner("PREOPERATIVE DIAGNOSIS:  ,Urinary hesitancy and weak stream.,POSTOPERATIVE DIAGNOSES:,1.  Urinary hesitancy and weak stream.,2.  Urethral narrowing.,3.  Mild posterior wall erythema.,PROCEDURE PERFORMED:,1.  Cystourethroscopy.,2.  Urethral dilation.,3.  Bladder biopsy and fulguration.,ANESTHESIA:  ,General.,SPECIMEN:  ,Urine culture sensitivity and cytology and bladder biopsy x1.,DISPOSITION: , To PACU in stable condition.,INDICATIONS AND FINDINGS:  ,This is a 76-year-old female with history of weak stream and history of intermittent catheterization secondary to hypotonic bladder in the past, last cystoscopy approximately two years ago.,FINDINGS AT TIME OF SURGERY:,  Cystourethroscopy revealed some mild narrowing of the urethra, which was easily dilated to #23 French.  A midureteral polyp was noted.  Cystoscopy revealed multiple cellules and mild trabeculation of the bladder.  Posterior wall revealed some mild erythema with some distorted architecture of the bladder mucosa.  No obvious raised bladder tumor was noted.  No foreign bodies were noted.  The ureteral orifices were noted on the trigone just proximal to the bladder neck.,DESCRIPTION OF PROCEDURE: , After informed consent was obtained, the patient was moved to the operating room, general anesthesia was induced by the Department of Anesthesia.  The patient was prepped and draped in normal sterile fashion and urethral sounds used to dilate the urethra to accommodate #23 French cystoscope.  Cystoscopy was performed in its entirety with the above findings.  The small area of erythema on the posterior wall was biopsied using a flexible biopsy forceps and Bovie cautery was used to cauterize and fulgurate this area.  The bladder was drained, cystoscope was removed, scope was reinserted and bladder was again reexamined.  No evidence of active bleeding noted.  The bladder was drained, cystoscope was removed, and the patient was cleaned and sent to recovery room in stable condition to followup with Dr. X in two weeks.  She is given prescription for Levaquin and Pyridium and given discharge instructions.")
note = str(doc_df.iloc[3810]['transcription'])
custom_doc = cmed_ner_nckk(note)
colors = {"SYMPTOM": "#938CE3", "DRUG": "#7DF6D9", "DISEASE":"#E39B8C"}
options = {"colors": colors}

spacy.displacy.render(custom_doc, style="ent", options= options, jupyter=True)

#Testing with model trained on Kaggle dataset (cmed_ner_kaggle)

In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/NLP NER Project/Models/cmed_ner_kaggle/model-best.zip" -d "/content"

In [ ]:
import spacy
cmed_ner_kaggle ="/content/content/model-best"
nc_nlp = spacy.load(cmed_ner_kaggle)

nc_nlp.component_names
nc_nlp.get_pipe('ner').labels

In [ ]:
nc_doc = nc_nlp(note)

#  print(ent.text," : ",ent.label_)
colors = {"PATHOGEN": "#F67DE3", "MEDICINE": "#7DF6D9", "MEDICALCONDITION":"#f9cb9c"}
options = {"colors": colors}

spacy.displacy.render(nc_doc, style="ent", options= options, jupyter=True)


#Testing with hugging face model

In [ ]:
!pip install https://huggingface.co/kormilitzin/en_core_med7_lg/resolve/main/en_core_med7_lg-any-py3-none-any.whl


In [ ]:
med7_nlp = spacy.load("en_core_med7_lg")

huggingface_doc = med7_nlp(note)
colors = {'DOSAGE': "#ea9999", "DRUG": "#f9cb9c","DURATION":"#b4a7d6","FORM":"#a2c4c9","FREQUENCY":"#b6d7a8","ROUTE":"#ebb6ff","STRENGTH":"#daf9ef"}
options = {"ents": ["DOSAGE","DRUG","DURATION","FORM","FREQUENCY","ROUTE","STRENGTH"], "colors": colors}

spacy.displacy.render(huggingface_doc,style ='ent',jupyter =True,options=options)




#Testing with Scispace model

In [ ]:
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bc5cdr_md-0.5.1.tar.gz

In [ ]:
sci_nlp = spacy.load("en_ner_bc5cdr_md")

scispcay_doc = sci_nlp(note)
colors = {'DISEASE': "#85C1EA", "CHEMICAL": "#ff6961"}
options = {"ents": ['DISEASE', 'CHEMICAL'], "colors": colors}

spacy.displacy.render(scispcay_doc,style ='ent',jupyter =True,options=options)

#Evaluating

In [ ]:
import re
doc = "I have a fever ,back pain and other diarrhea, Should I take medicine, tylonol? paracetamol or metformin?"
doc2 ="I am taking medicine, propanolol, rifampicin and quinolone.DO I have malaria or TB?"
pattern = 'fever|back pain|paracetamol|metformin|propanolol|rifampicin|quinolone|malaria|TB'
regex = re.compile(pattern)
diseases = regex.findall(doc2)
print("Extracting DISEASES: ",diseases)
for match in regex.finditer(doc2):
 print(match.group(), match.start(), match.end())


Extracting DISEASES:  ['propanolol', 'rifampicin', 'quinolone', 'malaria', 'TB']
propanolol 22 32
rifampicin 34 44
quinolone 49 58
malaria 69 76
TB 80 82


In [ ]:
import spacy
from spacy.scorer import Scorer
from spacy.tokens import Doc
from spacy.training.example import Example


examples = [
    ('I have a fever ,back pain and other diarrhea, Should I take medicine, tylonol? paracetamol or metformin?',
     {(9, 14, 'SYMPTOM'),(16, 25, 'SYMPTOM'),(79, 90, 'DRUG',(94, 103, 'DRUG'))}),
    ( ' I am taking medicine, propanolol, rifampicin and quinolone.DO I have malaria or TB?',
     {(22, 32, 'DRUG'), (34, 44, 'DRUG'),(49, 58, 'DRUG'),(59, 76, 'DISEASE'),(80, 82, 'DISEASE')})
]

def evaluate(ner_model, examples):
    scorer = Scorer()
    example = []
    for input_, annot in examples:
        pred = ner_model(input_)
        print(pred,annot)
        temp = Example.from_dict(pred, dict.fromkeys(annot))
        example.append(temp)
    scores = scorer.score(example)
    return scores

ner_model = cmed_ner_nckk
results = evaluate(ner_model, examples)
print(results)

I have a fever ,back pain and other diarrhea, Should I take medicine, tylonol? paracetamol or metformin? {(9, 14, 'SYMPTOM'), (16, 25, 'SYMPTOM'), (79, 90, 'DRUG', (94, 103, 'DRUG'))}
 I am taking medicine, propanolol, rifampicin and quinolone.DO I have malaria or TB? {(34, 44, 'DRUG'), (80, 82, 'DISEASE'), (22, 32, 'DRUG'), (49, 58, 'DRUG'), (59, 76, 'DISEASE')}
{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'sents_p': None, 'sents_r': None, 'sents_f': None, 'tag_acc': None, 'pos_acc': None, 'morph_acc': None, 'morph_micro_p': None, 'morph_micro_r': None, 'morph_micro_f': None, 'morph_per_feat': None, 'dep_uas': None, 'dep_las': None, 'dep_las_per_type': None, 'ents_p': None, 'ents_r': None, 'ents_f': None, 'ents_per_type': None, 'cats_score': 0.0, 'cats_score_desc': 'macro F', 'cats_micro_p': 0.0, 'cats_micro_r': 0.0, 'cats_micro_f': 0.0, 'cats_macro_p': 0.0, 'cats_macro_r': 0.0, 'cats_macro_f': 0.0, 'cats_macro_auc': 0.0, 'cats_f_per_type': {}, 'cats_auc_per_type

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nlp_ner.component_names
nlp_ner.get_pipe('ner').labels

('MEDICALCONDITION', 'MEDICINE', 'PATHOGEN')

In [ ]:
df = pd.read_csv('mtsamples.csv', index_col=0)
df.head()

In [ ]:
x = str(df.iloc[6]['transcription'])

docx = nlp_ner(x)

#  print(ent.text," : ",ent.label_)
colors = {"PATHOGEN": "#F67DE3", "MEDICINE": "#7DF6D9", "MEDICALCONDITION":"#FFFFFF"}
options = {"colors": colors}

spacy.displacy.render(docx, style="ent", options= options, jupyter=True)
results = [ent.text for ent in docx.ents if ent.label_ == 'MEDICALCONDITION' ]
results

['Lipodystrophy', 'Liposuction', 'ear']

#LAters


In [ ]:
import pandas as pd

In [ ]:
raw_df = pd.read_csv('mtsamples.csv', index_col=0)
raw_df.head()